# PhaseLab Quickstart: CRISPR Guide Design

This notebook demonstrates the core CRISPR guide design workflow in PhaseLab v0.6.1.

**Topics covered:**
- Designing guides for different modalities (CRISPRa, CRISPRi, Knockout)
- Understanding coherence scores and GO/NO-GO classification
- Filtering and ranking guides by multiple criteria
- Visualizing results

## Installation

```bash
pip install phaselab[plotting]
```

In [ ]:
import numpy as np
import pandas as pd

# Core PhaseLab imports
from phaselab import go_no_go
from phaselab.core.constants import E_MINUS_2

# CRISPR design imports
from phaselab.crispr import (
    design_guides,
    GuideConfig,
    Modality,
)

print(f"GO/NO-GO threshold (e^-2): {E_MINUS_2:.4f}")

## 1. Basic Guide Design

Let's design CRISPRa guides for the CLOCK gene, a core circadian rhythm regulator.

In [ ]:
# Configure the guide design
config = GuideConfig(
    target_gene="CLOCK",
    modality=Modality.CRISPRA,
    coherence_mode="heuristic",  # Fast mode for screening
    max_guides=20,
)

# Design guides
results = design_guides(config)

print(f"Designed {len(results.guides)} guides")
print(f"GO guides: {sum(1 for g in results.guides if g.is_go)}")
print(f"NO-GO guides: {sum(1 for g in results.guides if not g.is_go)}")

In [ ]:
# View top guides as DataFrame
df = results.to_dataframe()
df.head(10)

## 2. Understanding Scores

Each guide receives multiple scores that combine into the final ranking.

In [ ]:
# Examine a single guide in detail
top_guide = results.guides[0]

print(f"Sequence: {top_guide.sequence}")
print(f"PAM: {top_guide.pam}")
print(f"Position: {top_guide.position}")
print()
print("Score Breakdown:")
print(f"  GC Score: {top_guide.gc_score:.3f}")
print(f"  Thermo Score: {top_guide.thermo_score:.3f}")
print(f"  Position Score: {top_guide.position_score:.3f}")
print(f"  Specificity Score: {top_guide.specificity_score:.3f}")
print(f"  Coherence (R-bar): {top_guide.coherence:.3f}")
print()
print(f"Combined Score: {top_guide.combined_score:.3f}")
print(f"GO Status: {'GO' if top_guide.is_go else 'NO-GO'}")
print(f"Evidence Level: {top_guide.evidence_level}")

## 3. Filtering by Coherence

The GO/NO-GO threshold filters out unreliable guides.

In [ ]:
# Filter to only GO guides
go_guides = [g for g in results.guides if g.is_go]

print(f"GO guides: {len(go_guides)} / {len(results.guides)}")

# Check coherence distribution
coherences = [g.coherence for g in results.guides]
print(f"\nCoherence statistics:")
print(f"  Min: {min(coherences):.3f}")
print(f"  Max: {max(coherences):.3f}")
print(f"  Mean: {np.mean(coherences):.3f}")
print(f"  Threshold: {E_MINUS_2:.3f}")

In [ ]:
# Use go_no_go function directly
test_r_values = [0.05, 0.10, 0.135, 0.20, 0.50, 0.80]

print("R-bar    Status    Category")
print("-" * 35)
for r in test_r_values:
    result = go_no_go(r)
    print(f"{r:.3f}    {result.status:6}    {result.category}")

## 4. Comparing Modalities

Different CRISPR modalities have different optimal guide characteristics.

In [ ]:
# Design guides for each modality
modalities = [Modality.CRISPRA, Modality.CRISPRI, Modality.KNOCKOUT]
modality_results = {}

for mod in modalities:
    config = GuideConfig(
        target_gene="CLOCK",
        modality=mod,
        coherence_mode="heuristic",
        max_guides=10,
    )
    modality_results[mod.value] = design_guides(config)

# Compare top guide from each
print("Top Guide Comparison:")
print("=" * 60)
for mod_name, res in modality_results.items():
    top = res.guides[0]
    print(f"\n{mod_name}:")
    print(f"  Position: {top.position}")
    print(f"  Combined Score: {top.combined_score:.3f}")
    print(f"  Coherence: {top.coherence:.3f}")

## 5. Visualization

Visualize guide scores and positions.

In [ ]:
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Plot coherence vs combined score
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Scatter plot
    ax1 = axes[0]
    colors = ['green' if g.is_go else 'red' for g in results.guides]
    ax1.scatter(
        [g.coherence for g in results.guides],
        [g.combined_score for g in results.guides],
        c=colors,
        alpha=0.7,
        s=100,
    )
    ax1.axvline(E_MINUS_2, color='gray', linestyle='--', label=f'e^-2 = {E_MINUS_2:.3f}')
    ax1.set_xlabel('Coherence (R-bar)')
    ax1.set_ylabel('Combined Score')
    ax1.set_title('Coherence vs Combined Score')
    ax1.legend()
    
    # Position distribution
    ax2 = axes[1]
    positions = [g.position for g in results.guides]
    go_positions = [g.position for g in results.guides if g.is_go]
    nogo_positions = [g.position for g in results.guides if not g.is_go]
    
    ax2.hist(go_positions, bins=20, alpha=0.7, label='GO', color='green')
    ax2.hist(nogo_positions, bins=20, alpha=0.7, label='NO-GO', color='red')
    ax2.axvline(0, color='black', linestyle='-', label='TSS')
    ax2.set_xlabel('Position relative to TSS')
    ax2.set_ylabel('Count')
    ax2.set_title('Guide Position Distribution')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Install plotting dependencies: pip install phaselab[plotting]")

## 6. Export Results

Export guides for downstream analysis or ordering.

In [ ]:
# Export to CSV
df = results.to_dataframe()
df_go = df[df['is_go'] == True].head(5)

print("Top 5 GO guides for ordering:")
print(df_go[['sequence', 'pam', 'combined_score', 'coherence', 'evidence_level']])

# Save to file
# df_go.to_csv('CLOCK_crispra_guides.csv', index=False)

## 7. Using Quantum Mode (Optional)

For research-grade coherence assessment, use quantum mode with VQE.

In [ ]:
# Quantum mode requires additional dependencies
# pip install phaselab[quantum]

try:
    config_quantum = GuideConfig(
        target_gene="CLOCK",
        modality=Modality.CRISPRA,
        coherence_mode="quantum",  # VQE-based coherence
        max_guides=5,  # Fewer guides due to compute cost
    )
    
    results_quantum = design_guides(config_quantum)
    
    # Compare heuristic vs quantum coherence
    print("Quantum mode results:")
    for g in results_quantum.guides:
        print(f"  {g.sequence[:10]}... R={g.coherence:.3f} (Level {g.evidence_level})")
        
except ImportError:
    print("Quantum mode requires: pip install phaselab[quantum]")
except Exception as e:
    print(f"Quantum mode not available: {e}")

## Summary

Key takeaways:

1. **GO/NO-GO threshold (e^-2)**: Filters unreliable guides based on phase coherence
2. **Two-stage scoring**: Hard safety gates + soft ranking ensures both safety and efficacy
3. **Evidence levels**: A (hardware-validated) > B (quantum) > C (heuristic)
4. **Modality-specific optimization**: Different positions/features for CRISPRa vs CRISPRi vs Knockout

For more information:
- [Documentation](https://phaselab.readthedocs.io)
- [GitHub](https://github.com/followthesapper/phaselab)